In [1]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
text_df = pd.read_csv("/content/fake_or_real_news.csv")
text = list(text_df.text.values)
joined_text = " ".join(text)

with open("joined_text.txt", "w", encoding="utf-8") as f:
    f.write(joined_text)

In [3]:
partial_text = joined_text[:1000000]

In [4]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [5]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [6]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [7]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)  # for each sample, n input words and then a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)  # for each sample a boolean for each possible next word

In [8]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
# X_small = X[:5000]
# y_small = y[:5000]

# model.fit(X_small, y_small, batch_size=32, epochs=5, shuffle=True)


In [11]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

X_small = X[:5000]
y_small = y[:5000]

history = model.fit(X_small, y_small , batch_size=32, epochs=5, shuffle=True).history

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 73s 439ms/step - accuracy: 0.0312 - loss: 7.8852
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 87s 553ms/step - accuracy: 0.0479 - loss: 7.1038
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 120s 409ms/step - accuracy: 0.0499 - loss: 6.9542
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 66s 419ms/step - accuracy: 0.0587 - loss: 6.7438
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 79s 402ms/step - accuracy: 0.0672 - loss: 6.6378


In [12]:
history = model.fit(X_small, y_small, batch_size=32, epochs=5, shuffle=True).history

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 63s 400ms/step - accuracy: 0.0777 - loss: 6.4739
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 85s 417ms/step - accuracy: 0.0862 - loss: 6.2617
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 85s 437ms/step - accuracy: 0.1123 - loss: 6.0149
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 76s 398ms/step - accuracy: 0.1170 - loss: 5.7997
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 95s 483ms/step - accuracy: 0.1314 - loss: 5.5846


In [13]:
model.save("text_gen_model2.h5")
with open("history2.p", "wb") as f:
    pickle.dump(history, f)

In [14]:
model = load_model("text_gen_model2.h5")
history = pickle.load(open("history2.p", "rb"))

In [15]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1

    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [16]:
possible = predict_next_word("I will have to look into this thing because I", 5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 816ms/step


In [17]:
for idx in possible:
    print(unique_tokens[idx])

ve
have
america
new
be


In [18]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [19]:
generate_text("I will have to look into this thing because I", 100, 10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/s

'I will have to look into this thing because I was and more have know any it clinton ryan an eight who after who it the only setup but i fbi american march a contested convention for such more media made hillary and have country is re going and hate all like is going a sacred nature to her fbi trump to gone that is american you ayatollah the other republicans was his country trump and be been any the march values and an kgb for their families the last will election it clinton s eight was gone to those wake in our better act with ve up and my'

In [20]:
generate_text("The president of the United States announced yesterday that he", 100, 10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/

'The president of the United States announced yesterday that he country the kgb trump was had the hour filled my republicans the country of a fundamental hour the country election the united act is they own enough we hate enough to nationalist united states on i 11 the leadership filled this march to supporters more going on hillary to america it just have an kgb times on 11 york a meme for those march the burning symbols to those country on his personal march him have of a wake and that have going it fbi the clinton states had his last ayatollah in a family that have have going going'

In [23]:
for idx in predict_next_word("The president will most likely not be there to help", 10):
    print(unique_tokens[idx])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
was
to
trump
is
a
at
have
it
the
and


In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
